# Fill bottom layer of FastMECH 3D solution.
The FastMECH 3D solution is missing the bottom layer, the bed.  It only returns the logarithmic layers above the bed.  This results in an error in the particle-tracking code when the particles move into what would be the layer between the bed and the first node above the bed.  The code below simply fills in the bottom layer, with an elevation of the bed, and a velocity of zero.  

In [ ]:
import vtk
from vtk.util import numpy_support
import numpy as np
import os
print(os.getcwd())

In [ ]:
# # filenames of existing 2d and 3d solutions
# filename2d = '../data/NoStrmLnCurv_185cms2d1.vtk'
# filename3d = '../data/NoStrmLnCurv_185cms3d1.vtk'

# # filename of new 3d solution
# new3dsol = '../data/NoStrmLnCurv_185cms3d1_new.vtk'

# # filenames of existing 2d and 3d solutions
# filename2d = '../data/simple_meander/Result_FM_MEander_1_long_2D1.vtk'
# filename3d = '../data/simple_meander/Result_FM_MEander_1_long_3D1.vtk'

# # filename of new 3d solution
# new3dsol = '../data/simple_meander/Result_FM_MEander_1_long_3D1_new.vtk'

# # filenames of existing 2d and 3d solutions
# filename2d = '../data/IPC//NoStrmLnCurv_185cms2d1.vtk'
# filename3d = '../data/IPC/NoStrmLnCurv_185cms3d1.vtk'

# # filename of new 3d solution
# new3dsol = '../data/IPC/NoStrmLnCurv_185cms3d1_new.vtk'

# filenames of existing 2d and 3d solutions
filename2d = '../tests/data/Result_straight_2d_1.vtk'
filename3d = '../tests/data/Result_straight_3d_1.vtk'

# filename of new 3d solution
new3dsol = '../tests/data/Result_straight_3d_1_new.vtk'

In [ ]:
# Read existing solutions
vtksgrid2d = vtk.vtkStructuredGrid()
reader2d = vtk.vtkStructuredGridReader()
reader2d.SetFileName(filename2d)
reader2d.SetOutput(vtksgrid2d)
reader2d.Update()  # Needed because of GetScalarRange
output2d = reader2d.GetOutput()

vtksgrid3d = vtk.vtkStructuredGrid()
reader3d = vtk.vtkStructuredGridReader()
reader3d.SetFileName(filename3d)
reader3d.SetOutput(vtksgrid3d)
reader3d.Update()  # Needed because of GetScalarRange
output3d = reader3d.GetOutput()

In [ ]:
# Get dimensions of existing solutions
dims2d = output2d.GetDimensions()
dims3d = output3d.GetDimensions()
print(dims2d, dims3d)

In [ ]:
# Get 2d elevation which is the bottom boundary elevation of the 3d grid
Elevation_2D = vtksgrid2d.GetPointData().GetScalars("Elevation")
print(type(Elevation_2D)) # here to remind myself of what is returned

In [ ]:
# get the original velocity distribution
VelocityVec3D = vtksgrid3d.GetPointData().GetScalars("Velocity")
tmp = vtksgrid3d.GetPointData()

In [ ]:
na = tmp.GetNumberOfArrays()
for i in range(na):
    print(tmp.GetArrayName(i))

In [ ]:
tmp.GetNumberOfTuples()

In [ ]:
# Create the new structured grid.
dims = [dims3d[0], dims3d[1], dims3d[2]+1]
sgrid = vtk.vtkStructuredGrid()
sgrid.SetDimensions(dims)

# Create a writer to write the new solutions
writer = vtk.vtkStructuredGridWriter()
writer.SetFileName(new3dsol)

# Create the points (nodes) and new vectors (will be filled with bottom boundary). 
vectors = vtk.vtkDoubleArray()
vectors.SetNumberOfComponents(3)
vectors.SetName("Velocity")
vectors.SetNumberOfTuples(dims[0] * dims[1] * dims[2])
points = vtk.vtkPoints()
points.Allocate(dims[0] * dims[1] * dims[2])


In [ ]:

for k in range(0, dims3d[2]+1):
    koffset = k * dims3d[0] * dims3d[1]
    for j in range(0, dims3d[1]):
        joffset = j * dims3d[0]
        for i in range(0, dims3d[0]):
            offset = i + joffset + koffset
            offset2d = i + joffset
            xypt = vtksgrid2d.GetPoint(offset2d)
            elev = Elevation_2D.GetTuple(offset2d)
            
            # fill in boundary nodes when k == 0
            if k == 0:
#                 print(offset)
                vel = [0.0]*3
                # use xy locations and elevation from 2d solution which is the bottom boundary
                points.InsertPoint(offset, [xypt[0], xypt[1], elev[0]])
                # the velocity at the bed is zero
                vectors.InsertTuple(offset, (vel[0], vel[1], vel[2])) 
            else:
                # remove 1 whole k-layer from offset to get original solution
                xyzpt = vtksgrid3d.GetPoint(offset-(dims2d[0]*dims2d[1]))
                vel3d = VelocityVec3D.GetTuple(offset-(dims2d[0]*dims2d[1]))
                points.InsertPoint(offset, [xyzpt[0], xyzpt[1], xyzpt[2]])
                vectors.InsertTuple(offset, (vel3d[0], vel3d[1], vel3d[2]))
#                 print(offset, xypt, elev, vel3d)
                
# assign points and vectors to new grid                                 
sgrid.SetPoints(points)
sgrid.GetPointData().SetVectors(vectors)

# write new grid
writer.SetInputData(sgrid)
writer.Write()
                                            